# Analyse RFM – Feminiz

Ce notebook présente l'analyse RFM réalisée pour la marque fictive **Feminiz**.


**Objectifs :**

- Comprendre le comportement d'achat des clientes

- Calculer les indicateurs RFM (Récence, Fréquence, Montant)

- Segmenter les clientes en groupes **Low / Medium / High Value**

- Identifier des recommandations marketing actionnables


> 📝 Remarque : adapte le nom du fichier CSV et les noms de colonnes aux données que tu utilises réellement.


## 1. Import des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## 2. Chargement des données

On suppose un fichier CSV contenant les transactions avec, au minimum :

- `CustomerID` : identifiant client
- `InvoiceDate` : date de transaction
- `Amount` : montant de la transaction

Adapte ces noms de colonnes selon ta base.

In [ ]:
# 🔁 À adapter : remplace 'transactions_feminiz.csv' par ton fichier réel
file_path = "data/transactions_feminiz.csv"

df = pd.read_csv(file_path, parse_dates=["InvoiceDate"])
df.head()

## 3. Préparation des données

In [ ]:
df.info()

df = df.dropna(subset=["CustomerID", "InvoiceDate", "Amount"])

df = df[df["Amount"] > 0]

df.describe()

## 4. Calcul des indicateurs RFM

Nous allons calculer, pour chaque cliente :

- **Récence** : nombre de jours depuis le dernier achat
- **Fréquence** : nombre de transactions
- **Montant (Monetary)** : total dépensé


In [ ]:
reference_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)
reference_date

In [ ]:
rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": [lambda x: (reference_date - x.max()).days, "count"],
    "Amount": "sum"
})

rfm.columns = ["Recency", "Frequency", "Monetary"]

rfm.head()

## 5. Analyse descriptive des indicateurs RFM

In [ ]:
rfm.describe()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(rfm["Recency"], bins=30)
axes[0].set_title("Récence")

axes[1].hist(rfm["Frequency"], bins=30)
axes[1].set_title("Fréquence")

axes[2].hist(rfm["Monetary"], bins=30)
axes[2].set_title("Montant")

plt.tight_layout()
plt.show()

## 6. Création de scores RFM

In [ ]:
rfm["R_Score"] = pd.qcut(rfm["Recency"], 3, labels=[3, 2, 1])

rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 3, labels=[1, 2, 3])
rfm["M_Score"] = pd.qcut(rfm["Monetary"].rank(method="first"), 3, labels=[1, 2, 3])

rfm.head()

## 7. Score global et segmentation

In [ ]:
rfm["RFM_Score"] = rfm[["R_Score", "F_Score", "M_Score"]].astype(int).sum(axis=1)

def segment_from_score(score):
    if score <= 5:
        return "Low Value"
    elif score <= 7:
        return "Medium Value"
    else:
        return "High Value"

rfm["Segment"] = rfm["RFM_Score"].apply(segment_from_score)

rfm.head()

## 8. Analyse des segments

In [ ]:
segment_counts = rfm["Segment"].value_counts()
segment_counts

In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(segment_counts.index, segment_counts.values)
plt.title("Répartition des clientes par segment")
plt.xlabel("Segment")
plt.ylabel("Nombre de clientes")
plt.show()

In [ ]:
segment_value = rfm.groupby("Segment")[["Recency", "Frequency", "Monetary"]].mean()
segment_value

## 9. Conclusions et recommandations

- Les clientes `High Value` concentrent une grande partie du chiffre d'affaires.
- La fréquence d'achat est fortement liée au montant dépensé.
- Les clientes `Low Value` doivent être ciblées par des campagnes de réactivation.
- Possibilité de créer un programme VIP pour les `High Value`.

Tu peux adapter cette section en fonction de tes propres résultats et graphiques.